In [1]:
# 1. Configurações e preparações

# Importação das bibliotecas que serão utilizadas
import pandas as pd
import openpyxl
import os
from pandas.io.formats import excel

# Configurações para evitar formatação padrão dos cabeçalhos ao exportar o arquivo final e evitar também avisos de inconformidade de versão.
excel.ExcelFormatter.header_style = None
pd.set_option('chained_assignment', None)

# Caminho fonte de onde o arquivo estiver localizado no sistema.
# Ex: D:\Desafio_ISOps\Dados_consolidados.xlsx
CAMINHO_PARA_O_ARQUIVO_DADOS_CONSOLIDADOS = ''  

# Dataframes / Tabelas que serão utilizadas no processo
DADOS_CONSOLIDADOS = pd.read_excel(CAMINHO_PARA_O_ARQUIVO_DADOS_CONSOLIDADOS, sheet_name=['Engenharia', 'Marketing', 'Sales'])
dados_agregados = pd.DataFrame()

In [2]:
# 2. Agregar os dados em um(a) único Dataframe/Tabela

'''
Agrega os dados de todas as páginas num Dataframe. -- Um bom exemplo para entender como funciona na prática, 
                                                      poderia ser uma comparação de um merge/join usando SQL
'''

PAGINAS_PARA_UNIR = ['Engenharia', 'Marketing', 'Sales']

dados_agregados = pd.concat([DADOS_CONSOLIDADOS[pagina] for pagina in PAGINAS_PARA_UNIR]
                           ).reset_index(drop=True)

In [3]:
# 3. Realizar transformações nas colunas que possuem os valores do tipo Data para padronização no formato ISO8601

COLUNAS_PARA_FORMATAR_PARA_O_TIPO_DATA = ['Data_Candidatura', 'Data_Movimentacao']

''' 
A função verificar_tipo_de_dado_da_celula é usada para verificar se o tipo de dado de dada célula de uma coluna específica
está no formato datetime64[ns] (formato padrão do Excel para células do tipo Data).

Possíveis resultados:
Verdadeiro: O valor correto para atender ao critério de formatação ISO8601 é atribuido à célula.
Falso: Um valor nulo (NA) é atribuido à célula.
'''

def verificar_tipo_de_dado_da_celula(celula):
    if pd.api.types.is_datetime64_any_dtype(celula):
        return pd.NA if pd.isna(celula) else celula.date()
    else:
        return celula

for coluna in dados_agregados.columns[0:]:
    dados_agregados[coluna] = dados_agregados[coluna].apply(verificar_tipo_de_dado_da_celula)

In [4]:
# 4. Reescrever o arquivo final, adicionando uma página titulada de 'Todos', contendo os dados de todas as outras páginas.

with pd.ExcelWriter('Dados_Consolidados.xlsx', date_format='YYYY-MM-DD', engine='openpyxl', mode='a', if_sheet_exists = 'replace') as gravador:
    dados_agregados.to_excel(gravador, index=False, sheet_name='União')